<a href="https://colab.research.google.com/github/mille055/Rec_Project/blob/main/notebooks/Get_text_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Transformers to embed text columns
In this notebook we will embed the textual columns using document embeddings obtained using a pre-trained [Sentence Transformer](https://www.sbert.net) model.  SentenceTransformers is a framework for sentence / text embeddings which works particularly well for shorter text.  It was developed in 2019 and uses Siamese-BERT to develop semantically meaningful sentence embeddings which can be compared using cosine similarity.  You can use a [pretrained embedding model](https://www.sbert.net/docs/pretrained_models.html) or can train your own on a corpus.

**References:**
- [Sentence-BERT paper](https://arxiv.org/abs/1908.10084) by Reimers & Gurevych

In [32]:
import os
import numpy as np
import pandas as pd
import string
import time
import urllib.request
import zipfile
import torch

from sklearn.linear_model import LogisticRegression
!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
!pip install unidecode
import unidecode
import sys
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Download and prepare data

In [3]:
# Download the data
!git clone 'https://github.com/mille055/Rec_Project'

Cloning into 'Rec_Project'...
remote: Enumerating objects: 297, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 297 (delta 28), reused 26 (delta 10), pack-reused 246
Receiving objects: 100% (297/297), 122.16 MiB | 21.00 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Updating files: 100% (27/27), done.


In [4]:
podcast_df = pd.read_pickle('/content/Rec_Project/data/podcast_df_040423.pkl')
podcast_df

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id,rating,user
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,RobinFerris
1,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,1,Pops.99
2,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,5,ReddEye81
3,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,2,Keyta7777
4,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"Paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,[In celebration of our new premium format—two ...,1526579247,4,Okkupent
...,...,...,...,...,...,...,...,...,...,...,...,...
657194,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Monijansand
657195,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,trinityangel13
657196,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,Kweenkeys
657197,Quality Queen Control,Asha Christina,Education,"Sophistication, Psychology, Dating, and Lifest...",111,4.8,470.0,https://podcasts.apple.com/us/podcast/quality-...,[Hey Angels!!! In today's episode of the Quali...,1512702672,5,JoyfulJoyfulWOG


In [5]:
## clean text from the episode_descriptions column

sys.path.append('/content/Rec_Project/scripts')


import clean_dataframe_text
from clean_dataframe_text import join_and_clean_text, clean_text




In [6]:
### reducing the size of the dataframe prior to processing as do not need user ratings and userid for this
def prepare_df(df):
 
  
  df1= podcast_df.copy()
  # get rid of duplicates based on itunes_id
  cols_drop_dup = ['itunes_id']
  df_no_dups = df1.drop_duplicates(subset=cols_drop_dup)
  print('shape of new df without duplicates is ', df_no_dups.shape)
  # remove columns containing the user and user rating
  print('removing user  and rating columns')
  df_no_dups.drop(columns=['user', 'rating'], inplace=True)

  return df_no_dups



In [7]:
podcast_cleaned_df = prepare_df(podcast_df)

# clean the podcast dataframe
podcast_cleaned_df.episode_descriptions = podcast_cleaned_df.episode_descriptions.apply(join_and_clean_text)
podcast_cleaned_df.description = podcast_cleaned_df.description.apply(clean_text)


shape of new df without duplicates is  (3936, 12)
removing user  and rating columns


In [8]:
podcast_cleaned_df

,title,producer,genre,description,num_episodes,avg_rating,num_reviews,link,episode_descriptions,itunes_id
0,One Strange Thing: Paranormal & True-Weird Mys...,One Strange Thing,History,"paranormal, unexplainable, and uncanny stories...",105,4.6,499.0,https://podcasts.apple.com/us/podcast/one-stra...,in celebration of our new premium format two ...,1526579247
100,BibleProject,BibleProject Podcast,Religion & Spirituality,the creators of bibleproject have in depth con...,352,4.9,15000.0,https://podcasts.apple.com/us/podcast/biblepro...,"david was israel's greatest king, but even he ...",1050832450
200,The Domonique Foxworth Show,ESPN,Sports,with episodes every tuesday and thursday durin...,70,4.9,1100.0,https://podcasts.apple.com/us/podcast/the-domo...,"domonique, charlie, and ashley foxworth along ...",1642566714
300,Hacking Humans,CyberWire Inc.,Technology,"deception, influence, and social engineering i...",415,4.7,255.0,https://podcasts.apple.com/us/podcast/hacking-...,"kathleen smith, cmo from clearedjobs.net sits ...",1391915810
400,Leader Up,AMSC,Government,"leader up, a podcast by the army management st...",52,5.0,14.0,https://podcasts.apple.com/us/podcast/leader-u...,msc's mr. david howey meets with csm jason c. ...,1378682853
...,...,...,...,...,...,...,...,...,...,...
656189,Tales from the Stinky Dragon,Rooster Teeth,Leisure,a d amp;d podcast from rooster teeth! our brav...,101,4.9,781.0,https://podcasts.apple.com/us/podcast/tales-fr...,"with asafee on his deathbed, the four chosen o...",1563814788
656589,Morning Microdose,Almost 30,Education,the fact that you came across morning microdos...,159,5.0,187.0,https://podcasts.apple.com/us/podcast/morning-...,drop in for this mind expanding conversation w...,1639123211
656989,Presidential,Washington Post Audio,History,the washington post's presidential podcast exp...,52,4.4,3500.0,https://podcasts.apple.com/us/podcast/presiden...,"students, teachers and historians reflect on w...",1072170823
657089,Badlands Cola | A Strange Audio Drama,Renee Taylor Klint,Fiction,badlands cola is a cinematic mystery horror au...,17,4.6,63.0,https://podcasts.apple.com/us/podcast/badlands...,"hi listeners! it's renee, and today we're doin...",1627191206


In [11]:
podcast_cleaned_df.description.tolist()[0]

"paranormal, unexplainable, and uncanny stories aren't just in the fiction section. they happen every day, to people just like you. one strange thing brings you family friendly stories from america's newspaper archives. and they all have something in common: an element that can't be explained by logi..."

In [12]:
podcast_cleaned_df.to_pickle('cleaned_df.pkl')

## Create document embeddings
We will load a pre-trained model [('all-MiniLM-L6-v2')](https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2) which we will then use to create embeddings for our training and test set text.  The MiniLM-L6-v2 model was trained on 1.1 billion sentence pairs to produce high-quality sentence / short document embeddings in 384 dimensions which can be used for example to calculate similarity between documents.  

In [13]:
# Load pre-trained model
senttrans_model = SentenceTransformer('all-MiniLM-L6-v2',device=device)



In [14]:
# Create embeddings for columns description, episode descriptions, genre
def create_embeddings(df, cols):
  df1 = df.copy()
  
  new_col_names = []
  for col in cols:
    print('Now embedding column', col)
    col_data = df1[col].values.tolist()
    col_embeds = [senttrans_model.encode(doc) for doc in col_data]
    new_col_name = col + '_embedding'
    df1[new_col_name] = col_embeds
    new_col_names.append(new_col_name)

  embeddings_df = df1[new_col_names]
  embeddings_df['itunes_id'] = df1['itunes_id']

  return df1, embeddings_df



In [15]:
podcast_with_embeds, embeddings_only = create_embeddings(podcast_cleaned_df, cols= ['description', 'genre','episode_descriptions'])


Now embedding column description
Now embedding column genre
Now embedding column episode_descriptions


In [17]:
podcast_with_embeds.to_pickle('podcast_base_with_embeds.pkl')

In [18]:
embeddings_only.to_pickle('podcast_embeddings_only.pkl')

In [19]:
embeddings_only.columns

Index(['description_embedding', 'genre_embedding',
       'episode_descriptions_embedding', 'itunes_id'],
      dtype='object')

## Cosine similarity of the embeddings

Finally, we will used our embeddings as features to train a softmax regression model to classify the documents.

In [20]:
embeddings_only.columns

Index(['description_embedding', 'genre_embedding',
       'episode_descriptions_embedding', 'itunes_id'],
      dtype='object')

In [63]:
# combined_features = np.hstack(embeddings_only[['description_embedding', 'genre_embedding',
#        'episode_descriptions_embedding']].values)

In [21]:
def create_cosine_similarity(df, feats = ['genre_embedding', 'description_embedding', 'episode_descriptions_embedding']):
  array_list = []
  for feat in feats:
    array_list.append(np.stack(df[feat].values))
  concat_array = np.concatenate((array_list), axis=1)
  print('after concatenate, data size is ', concat_array.shape)
  matrix = cosine_similarity(concat_array)
  
  return matrix



In [29]:
cs_all = create_cosine_similarity(embeddings_only)
cs_genre = create_cosine_similarity(embeddings_only, feats=['genre_embedding'])
cs_desc = create_cosine_similarity(embeddings_only, feats=['description_embedding'])
cs_episo = create_cosine_similarity(embeddings_only, feats=['episode_descriptions_embedding'])
cs_gen_desc = create_cosine_similarity(embeddings_only, feats=['genre_embedding', 'description_embedding'])


after concatenate, data size is  (3936, 1152)
after concatenate, data size is  (3936, 384)
after concatenate, data size is  (3936, 384)
after concatenate, data size is  (3936, 384)
after concatenate, data size is  (3936, 768)


## Generate predicted ratings
Ready to generated predicted ratings for each user-item pair.  The process we will use to generate each predicted rating is as follows:  
- Filter the similarity matrix to only the movies previously watched by the user  
- Find the previously watched movie that is most similar to the movie for which we want to generate the predicted rating (nearest neighbor approach)
- Get the user's rating for the most similar previously watched movie and use that as our prediction

In [31]:
# Split our data into training and validation sets
from sklearn.model_selection import train_test_split
X = podcast_df[['user', 'itunes_id']]
y = podcast_df['rating']
X_train, X_val, y_train, y_val = train_test_split(X,y,random_state=0, test_size=0.2)

In [38]:
# First, we'll use the cosine similarity of the genre and description features
sim_matrix = pd.DataFrame(cs_gen_desc)
sim_matrix.head()


,0,1,2,3,4,5,6,7,8,9,...,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935
0,1.000000,0.213808,0.305327,0.342635,0.273094,0.238568,0.601837,0.215632,0.271587,0.274726,...,0.267439,0.339208,0.223705,0.269134,0.290784,0.205596,0.372644,0.499983,0.281771,0.350177
1,0.213808,1.000000,0.346491,0.114829,0.283784,0.348318,0.266916,0.244482,0.284012,0.067016,...,0.327292,0.206715,0.335630,0.324960,0.209102,0.231479,0.193162,0.236213,0.177488,0.308920
2,0.305327,0.346491,1.000000,0.188584,0.324291,0.368471,0.297782,0.300338,0.311772,0.162914,...,0.354254,0.331577,0.338124,0.364951,0.633313,0.354219,0.267382,0.336096,0.266566,0.391271
3,0.342635,0.114829,0.188584,1.000000,0.252362,0.182837,0.232414,0.229940,0.291919,0.199730,...,0.330243,0.360145,0.256898,0.241604,0.202092,0.202132,0.310288,0.189712,0.259304,0.332494
4,0.273094,0.283784,0.324291,0.252362,1.000000,0.343525,0.319082,0.263709,0.329524,0.287968,...,0.420794,0.214495,0.285665,0.317171,0.268542,0.311570,0.230132,0.434224,0.220061,0.409019


In [39]:
def predict_rating(user_item_pair,simtable = sim_matrix,X_train=X_train, y_train=y_train):
    podcast_to_rate = user_item_pair['itunes_id']
    user = user_item_pair['user']
    # Filter similarity matrix to only podcasts already reviewed by user
    prior_podcast = X_train.loc[X_train['user']==user, 'itunes_id'].tolist()
    simtable_filtered = simtable.loc[podcast_to_rate, prior_podcast]
    # Get the most similar movie already watched to current podcast to rate
    most_similar = simtable_filtered.index[np.argmax(simtable_filtered)]
    # Get user's rating for most similar podcast
    idx = X_train.loc[(X_train['user']==user) & (X_train['itunes_Id']==most_similar)].index.values[0]
    most_similar_rating = y_train.loc[idx]
    return most_similar_rating

In [40]:
# Get the predicted ratings for each podcast in the validation set and calculate the RMSE
ratings_valset = X_val.apply(lambda x: predict_rating(x),axis=1)
val_rmse = np.sqrt(mean_squared_error(y_val,ratings_valset))
print('RMSE of predicted ratings is {:.3f}'.format(val_rmse))

KeyError: ignored

## Evaluate model performance

Accuracy on the test set is 0.896
